In [1]:
import numpy as np
from typing import Dict, Tuple, List
from copy import copy
from collections import defaultdict
from heapq import *

In [2]:
class Network:
    def __init__(
        self,
        nodes: List[object],
        edges: List[Tuple[object, object]],
        capacities: List[int],
        costs: List[int],
        supplies: List[int] 
    ) -> None:
        
        M = 100000
        self.U = int(np.max(np.array(capacities)))
        
        # Edge representation: (origin, destination, unique_id)
        # Even if network is not, residual graph is a multigraph 
        # so we need unique_id to determine which edge to use.
        
        # Update input for dummy nodes
        edges = [
            *edges,
            *[(i, -1) for i in nodes],
            *[(-2, i) for i in nodes],
            (-1, -2)
        ]

        capacities = [
            *capacities,
            *[M for _ in range(2 * len(nodes) + 1)]
        ]

        costs = [
            *costs,
            *[0 for _ in range(2 * len(nodes))],
            M
        ]

        supplies = [*supplies, 0, 0]

        self.V = [*nodes, -1, -2]
        self.E = [(*edges[i-1], i) for i in range(1, len(edges) + 1)]
        self.u = dict(zip(self.E, capacities))
        self.c = dict(zip(self.E, costs))
        self.b = dict(zip(self.V, supplies))

In [3]:
# Adapted from https://gist.github.com/kachayev/5990802

def rev(edge):
    return (edge[1], edge[0], -edge[2])

def dijkstra(
    s: object,
    t: object,
    adj: Dict[Tuple[object, object], int]
) -> Tuple[Dict[object, int], List[Tuple[object, object]]]:
    '''
    Computes the shortest path distances from [s] to all other nodes 
    and the shortest path from [s] to [t] on the graph with edges/weights 
    given by the entries of [adj]. Implementation of Dijkstra's shortest
    path algorithm for graphs with non-negative edge costs.
    
    Args:
        s: Start node
        t: End node (for path)
        adj: Dictionary containing edges and their respective weights, with
            adj[(i, j)] = w_{ij} for nodes i and j.
    '''
    def _format_path(lst):
        return [(lst[i][0], *lst[i+1]) for i in range(len(lst) - 1)]
            
            
    # Generate underlying adjacency lists
    adjacency = defaultdict(list)
    for (i, j, id), c in adj.items():
        if c < 0:
            print(f"cost can't be < 0: {c}")
        assert(c >= 0)
        adjacency[i].append((j, id, c))

    # Initialize queue, distances
    queue, seen, distances = [(0, s, 0, [])], set(), {s: 0}
    
    while queue:
        cost, v1, id, path = heappop(queue)
        if v1 not in seen:
            seen.add(v1)
            path = [*path, (v1, id)]
            if v1 == t: 
                out_path = _format_path(path)

            for v2, id, c in adjacency.get(v1, ()):
                if v2 in seen: 
                    continue
                prev = distances.get(v2, None)
                nxt = cost + c
                if prev is None or nxt < prev:
                    distances[v2] = nxt
                    heappush(queue, (nxt, v2, id, path))
    pp = [(e[0], e[1]) for e in out_path]
    print(f"P={pp}") 
    return distances, out_path

In [4]:
def reduced_cost(
    N: Network,
    u_f: Dict[Tuple[object, object], int],
    p: Dict[object, int]
) -> Dict[Tuple[object, object], int]:
    '''
    Computes reduced costs of the edges in the residual graph [u_f] with respect to edge
    costs [N.c] and node potentials [p].
    
    Args:
        N: Network representing the problem input
        u_f: Dictionary encoding the residual graph w.r.t the current flow
        p: Current node potentials
        
    Returns:
        A dictionary which gives the reduced cost for each edge (u, v) according to
        c_p[(u, v)] = c[(u, v)] + p[u] - p[v].
    '''
    reduced_costs = {}
    for e in u_f.keys():
        (u, v, _) = e
        if e in N.c:
            reduced_costs[e] = N.c[e] + p[u] - p[v]
        else:
            reduced_costs[e] = -N.c[rev(e)] + p[u] - p[v]
    return reduced_costs
    
def excess_nodes(
    N: Network,
    f: Dict[Tuple[object, object], int],
    K: int
) -> Tuple[List, List]:
    '''
    Compute nodes in the network [N] where flow conservation is violated by at least [K] units
    for the flow [f]. 
    
    Args:
        N: Network object representing the problem input
        f: Potentially infeasible flow
        K: Scaling parameter
    
    Returns:
        A tuple consisting of a list of nodes where net flow in is greater than [K]
        and a list of nodes where the net flow in is less than -[K].
    '''
    def _excess(N, f) -> Dict[object, int]:
        # Initialize excess to be supply
        excess = {v: N.b[v] for v in N.V}
        for (u, v, _), val in f.items():
            excess[u] -= val
            excess[v] += val

        return excess
    
    e_f = _excess(N, f)
    S_f = [i for (i, val) in e_f.items() if val >= K]
    T_f = [i for (i, val) in e_f.items() if val <= -K]
    return S_f, T_f

def update_potentials(
    p: Dict[object, int],
    distances: Dict[object, int]
) -> None:
    '''
    Updates node potentials [p] with the shortest path distances in
    [distances] according to p[i] <- p[i] + distances[i].
    
    Args:
        p: Previous node potentials
        distances: Shortest path distance to each node in graph from a node with
            surplus above the current scaling threshold
        
    '''
    for i in p.keys():
        p[i] += distances[i]
                   
def saturate_edges(
    N: Network,
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int],
    edges: List[Tuple[object, object]]
) -> None:
    '''
    Updates the flow [f] and residual graph [u_f] by saturating
    all edges in [edges].
    
    Args:
        N: Flow network encoding the problem input
        u_f: The residual graph for the current flow
        f: The current flow
        edges: List of edges to saturate
        
    '''
    for e in edges:
        if e in f:
            f[e] = N.u[e]                              # Saturate foward edge
            u_f[e] = 0                                 # Zero forward residual edge
            u_f[rev(e)] = N.u[e]                       # Saturate backward residual edge

        else:
            f[rev(e)] = 0                              # Zero forward edge
            u_f[e] = 0                                 # Saturate forward residual edge
            u_f[rev(e)] = N.u[rev(e)]                  # Zero backward residual edge

        
def saturate_neg_cost_admissible(
    N: Network,
    c_p: Dict[Tuple[object, object], int],
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int],
    K: int
) -> None:
    '''
    Updates the current flow [f] and residual graph [u_f] by
    saturating all edges with residual capacity of at least [K]
    and negative reduced cost [c_p] to preserve invariants in the
    algorithm.
    
    Args:
        N: Flow network encoding the problem input
        c_p: Current reduced costs
        f: The current flow
        u_f: The residual graph for the current flow
        K: Scaling parameter
    '''
    neg_cost_admissible = [
        e
        for e, u in u_f.items() 
        if u >= K and c_p[e] < 0
    ]
    print(f"Number of negative cost admissible edges: {len(neg_cost_admissible)}")
    
    saturate_edges(N, f, u_f, neg_cost_admissible)
    
    # TODO remove
    for e,c in c_p.items():
        if c < 0 and u_f[e] >= K:
            print(f"capacity of the negative reduced cost edge w. cap < K: {u_f[e]}, and cost {c}")
            assert(u_f[e] < K)
    
def augment_flow_along_path(
    P: List[Tuple[object, object]],
    f: Dict[Tuple[object, object], int],
    u_f: Dict[Tuple[object, object], int],
    K: int
) -> None:
    ''' 
    Updates the current flow [f] and residual graph [u_f] by
    pushing [K] units of flow along the directed path P.
    
    Args:
        P: Path of edges to push flow
        f: Current flow
        c_f: Current residual graph
        K: Scaling parameter
    
    '''
    for e in P:
        if e in f:
            f[e] += K
            u_f[e] -= K
            u_f[rev(e)] = u_f.get(rev(e), 0) + K
            
        else:
            f[rev(e)] -= K
            u_f[rev(e)] += K
            u_f[e] -= K

def value(N, f):
    return np.sum([f[e]*N.c[e] for e in N.E])
    
def capacity_scaling(
    N: Network,
    p: Dict[object, int]
) -> Tuple[Dict[Tuple[object, object], int], Dict[object, int]]:
    '''
    Primal-dual algorithm for computing a minimum-cost flow for the 
    network [N] starting from dual-feasible node potentials [p].
    
    Args:
        N: Flow network encoding the problem input
        p: Initial node potentials for warm start
        
    Returns:
        Minimum cost flow and corresponding optimal node potentials
    '''
    
    # Init zero flow and potentials
    f = {e: 0 for e in N.E}
    K = N.U
    u_f = copy(N.u)
    p = copy(p)
    iters = 0
    
    while K >= 1:
        print(f"iteration: {iters}, flow value: {value(N, f)}, p: {p.get(8635, -1)}")
        # Compute reduced costs w.r.t potentials p
        c_p = reduced_cost(N, u_f, p)

        # Saturate admissible edges with negative reduced cost
        saturate_neg_cost_admissible(N, c_p, f, u_f, K)
   
        # Compute new admissible edges, and surplus/deficit nodes above scaling threshold
        S_f, T_f = excess_nodes(N, f, K)  
        
        while len(S_f) > 0 and len(T_f) > 0:
            s = S_f[0]
            t = T_f[0]
            
            # TODO remove
            #for e,c in c_p.items():
            #    if c < 0 and u_f[e] >= K:
                    #print(f"capacity of the negative reduced cost edge w. cap < K: {u_f[e]}, and cost {c}")
                    #assert(u_f[e] < K)
                    
            
            # Admissible edges
            adj = {e: c for (e, c) in c_p.items() if u_f[e] >= K}
            D, P = dijkstra(s, t, adj)
            update_potentials(p, D)
            augment_flow_along_path(P, f, u_f, K)
            c_p = reduced_cost(N, u_f, p)
            # TODO remove?? Saturate admissible edges with negative reduced cost
            saturate_neg_cost_admissible(N, c_p, f, u_f, K)
            S_f, T_f = excess_nodes(N, f, K) 
            iters += 1

        K //= 2
        
    print(f"Number of flow updates: {iters}")
    return f, p

In [5]:
edges = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,2)]
capacities = np.array([15, 8, 20, 4, 10, 15, 4, 20, 5])
costs = np.array([4, 4, 2, 2, 6, 1, 3, 2, 3])
supplies = [20, 0, 0, -5, -15]
nodes = [0, 1, 2, 3, 4]
N = Network(nodes, edges, capacities, costs, supplies)     

p = {v: 0 for v in N.V}
f, p = capacity_scaling(N, p)

iteration: 0, flow value: 0, p: -1
Number of negative cost admissible edges: 0
iteration: 0, flow value: 0, p: -1
Number of negative cost admissible edges: 0
P=[(0, 1), (1, 2), (2, 3), (3, 4)]
Number of negative cost admissible edges: 0
iteration: 1, flow value: 90, p: -1
Number of negative cost admissible edges: 1
P=[(2, 3)]
Number of negative cost admissible edges: 0
iteration: 2, flow value: 127, p: -1
Number of negative cost admissible edges: 1
P=[(0, 1)]
Number of negative cost admissible edges: 0
P=[(2, 1)]
Number of negative cost admissible edges: 0
P=[(3, 4)]
Number of negative cost admissible edges: 0
P=[(3, 4)]
Number of negative cost admissible edges: 0
iteration: 6, flow value: 147, p: -1
Number of negative cost admissible edges: 0
P=[(2, 4)]
Number of negative cost admissible edges: 0
Number of flow updates: 7


In [6]:
def feasibility_check(N, f):
    assert np.all(np.array([len(excess_nodes(N, f, 0.00001)[i]) == 0 for i in [0,1]]))
    assert np.all(np.array(list(f.values())) >= 0)
    assert np.all(np.array(list(f.values())) <= np.array(list(N.u.values())))
    
def optimality_check(N, f, p):
    primal = np.sum([f[e]*N.c[e] for e in N.E])
    dual = -np.sum([p[i] * N.b[i] for i in N.V]) - np.sum([N.u[e] * max(0, p[e[1]] - p[e[0]] - N.c[e]) for e in N.E])
    assert np.isclose(primal, dual)
    return primal

In [7]:
feasibility_check(N, f)
optimality_check(N, f, p)

150

In [78]:
def parse(filename) -> Network:
    """
    Parses a network file following the DIMACS problem specification 
    structure and transforms it into a Network object
    
    Some elements of the specification:
    - Lines starting in c are comments
    - Lines starting in p explain what problem to solve (can be ignored, 
      we only consider minimum-cost flow problems)
    - Lines starting in n define nodes
    - Lines starting in a define arcs (edges)
    
    Args:
        filename: name of the file containing the network data
        
    Returns:
        The corresponding Network object
    """
    # Lines we can ignore
    ignore_list = ['c', 'p']
    
    file = open(filename, 'r')
    
    # Nodes is a hashmap from node values to their supply
    nodes = {}
    # Edges is a hashmap from edges to a tuple with their capacity and cost
    edges = {}
    
    for line in file:
        if len(line) > 0 and line[0] not in ignore_list:
            if line[0] == 'n':
                # Node parsing
                node = [int(elem) for elem in line.split(' ')[1:]]
                nodes[node[0]] = node[1]
            elif line[0] == 'a':
                arc = [int(elem) for elem in line.split(' ')[1:]]
                node1 = arc[0]
                node2 = arc[1]
                capacity = arc[3]
                cost = arc[4]
                
                # Only nodes with non-zero supply are in a "node line"
                if node1 not in nodes:
                    nodes[node1] = 0
                if node2 not in nodes:
                    nodes[node2] = 0
                if (node1, node2) in edges:
                    # TODO not amazing (reaverages every time)
                    old_capacity, old_cost = edges[(node1, node2)]
                    new_cost = old_cost * old_capacity + cost * capacity
                    new_cost /= (old_capacity + capacity)
                    edges[(node1, node2)] = (old_capacity + capacity, new_cost)
                else:
                    edges[(node1, node2)] = (capacity, cost)
    file.close()
    
    capacities, costs = zip(*edges.values())
    network = Network(nodes.keys(), edges.keys(), capacities, costs, nodes.values()) 
    
    print(f"This dataset contains: {len(nodes.keys())} nodes and {len(edges.keys())} edges")

    return network

In [ ]:
network = parse("resources/road_flow_01_DC_a.txt")
p2 = {v: 0 for v in network.V}
f, _ = capacity_scaling(network, p2)

This dataset contains: 9559 nodes and 29682 edges
iteration: 0, flow value: 0.0, p: 0
Number of negative cost admissible edges: 0
P=[(7822, -1), (-1, -2), (-2, 7962)]
Number of negative cost admissible edges: 0
P=[(5748, -1), (-1, -2), (-2, 6614)]
Number of negative cost admissible edges: 0
P=[(3282, -1), (-1, -2), (-2, 6614)]
Number of negative cost admissible edges: 0
P=[(3282, -1), (-1, -2), (-2, 3871)]
Number of negative cost admissible edges: 0
P=[(2760, -1), (-1, -2), (-2, 3233)]
Number of negative cost admissible edges: 0
P=[(768, -1), (-1, -2), (-2, 2137)]
Number of negative cost admissible edges: 0
P=[(768, -1), (-1, -2), (-2, 1682)]
Number of negative cost admissible edges: 0
iteration: 7, flow value: 672000000.0, p: 500000
Number of negative cost admissible edges: 16
P=[(8635, 8637), (8637, 8640), (8640, 6448), (6448, 6446), (6446, 6444), (6444, 6445), (6445, 6430), (6430, 6431), (6431, 6427), (6427, 6425), (6425, 6426), (6426, 6424), (6424, 6423), (6423, 6421), (6421, 6420)

P=[(3265, 4263), (4263, 4261), (4261, 4262), (4262, 4260), (4260, 4071), (4071, 4072), (4072, 4069), (4069, 4062), (4062, 4052), (4052, 4013), (4013, 4010), (4010, 4002), (4002, 4006), (4006, 4004), (4004, 4001), (4001, 3865), (3865, 3862), (3862, 3860), (3860, 3844), (3844, 3843), (3843, 3842), (3842, 3874), (3874, 3845), (3845, 3863), (3863, 3867), (3867, 3892), (3892, 4022), (4022, 4018), (4018, 4027), (4027, 4035), (4035, 4036), (4036, 4026), (4026, 3897), (3897, 3893), (3893, 3889), (3889, 3876), (3876, 3872), (3872, 3871), (3871, -2), (-2, 6614), (6614, 6586), (6586, 6585), (6585, 6588), (6588, 6589), (6589, 6615), (6615, 6619), (6619, 6661), (6661, 6590), (6590, 6636), (6636, 6634), (6634, 6635), (6635, 6637), (6637, 6633), (6633, 6629), (6629, 6534), (6534, 6530), (6530, 6529), (6529, 6526), (6526, 6523), (6523, 6536), (6536, 6538), (6538, 6535), (6535, 6540), (6540, 6703), (6703, 6700), (6700, 6701), (6701, 6708), (6708, 6705), (6705, 6706), (6706, 6717), (6717, 6718), (6718, 

P=[(414, 413), (413, 416), (416, 412), (412, 415), (415, 417), (417, 420), (420, 438), (438, 440), (440, 441), (441, 794), (794, 797), (797, 800), (800, 801), (801, 803), (803, 804), (804, 874), (874, 893), (893, 894), (894, 902), (902, 903), (903, 905), (905, 908), (908, 910), (910, 924), (924, 925), (925, 958), (958, 1999), (1999, 1079), (1079, 1077), (1077, 1081), (1081, 1083), (1083, 1102), (1102, 1101), (1101, 1104), (1104, 1112), (1112, 2010), (2010, 2011), (2011, 1113), (1113, 1106), (1106, 1103), (1103, 1085), (1085, 1069), (1069, 1067), (1067, 970), (970, 1066), (1066, 974), (974, 1070), (1070, 975), (975, 1073), (1073, 1297), (1297, 1298), (1298, 1300), (1300, 1175), (1175, 1176), (1176, 1301), (1301, 1306), (1306, 1314), (1314, 1316), (1316, 1321), (1321, 1335), (1335, 1357), (1357, 1359), (1359, 1361), (1361, 1372), (1372, 1425), (1425, 2289), (2289, 2293), (2293, 2298), (2298, 2291), (2291, 1442), (1442, 1439), (1439, 2311), (2311, 2315), (2315, 2317), (2317, 3769), (3769,

Number of negative cost admissible edges: 0
P=[(5730, 5723), (5723, 5722), (5722, 3387), (3387, 3386), (3386, 3385), (3385, 3388), (3388, 3365), (3365, 3364), (3364, 3360), (3360, 3359), (3359, 3207), (3207, 3202), (3202, 3182), (3182, 3181), (3181, 3142), (3142, 3136), (3136, 3137), (3137, 3138), (3138, 3139), (3139, 3147), (3147, 3231), (3231, 3232), (3232, 3234), (3234, 3240), (3240, 3275), (3275, 3276), (3276, 3307), (3307, 3310), (3310, 3314), (3314, 3316), (3316, 3331), (3331, 3334), (3334, 3428), (3428, 3335), (3335, 3435), (3435, 3417), (3417, 3423), (3423, 3426), (3426, 3460), (3460, 3480), (3480, 3503), (3503, 3505), (3505, 3507), (3507, 5819), (5819, 5837), (5837, 1379), (1379, 1378), (1378, 5838), (5838, 3508), (3508, 5836), (5836, 5840), (5840, 5841), (5841, 5843), (5843, 5842), (5842, 4568), (4568, 4569), (4569, 4552), (4552, 4549), (4549, 4499), (4499, 3470), (3470, 3456), (3456, 4488), (4488, 4485), (4485, 4489), (4489, 4487), (4487, 4484), (4484, 4355), (4355, 4482), (

P=[(4659, 4644), (4644, 4645), (4645, 4661), (4661, 4663), (4663, 4658), (4658, 4657), (4657, 4655), (4655, 4653), (4653, 4649), (4649, 4648), (4648, 4650), (4650, 4647), (4647, 4634), (4634, 4627), (4627, 4635), (4635, 4636), (4636, 4633), (4633, 4628), (4628, 4621), (4621, 4618), (4618, 4609), (4609, 4606), (4606, 4523), (4523, 4539), (4539, 4537), (4537, 4515), (4515, 4513), (4513, 4512), (4512, 4505), (4505, 4496), (4496, 4494), (4494, 4491), (4491, 4487), (4487, 4489), (4489, 4485), (4485, 4488), (4488, 3456), (3456, 3470), (3470, 4499), (4499, 4549), (4549, 4552), (4552, 4569), (4569, 4568), (4568, 5842), (5842, 5843), (5843, 5841), (5841, 5840), (5840, 5836), (5836, 3508), (3508, 5838), (5838, 1378), (1378, 1379), (1379, 5837), (5837, 5819), (5819, 3507), (3507, 3505), (3505, 3503), (3503, 3480), (3480, 3460), (3460, 3426), (3426, 3423), (3423, 3417), (3417, 3435), (3435, 3335), (3335, 3428), (3428, 3334), (3334, 3331), (3331, 3316), (3316, 3314), (3314, 3310), (3310, 3307), (33

...

P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number 

P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number 

P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number 

Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost 

P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number of negative cost admissible edges: 1
P=[(7981, 7978)]
Number 

In [ ]:
feasibility_check(network, f)
optimality_check(network, f, p)

In [ ]:
f, p = capacity_scaling(network, p)

In [ ]:
test1 = {(e[0], e[1]): network.c[e] + p[e[0]] - p[e[1]] for e in network.E}
p2 = {v: 0 for v in network.V}
test2 = {(e[0], e[1]): network.c[e] + p2[e[0]] - p2[e[1]] for e in network.E}

In [ ]:
len([0 for e in test1.items() if e[1] < 0])

In [ ]:
len([0 for e in test2.items() if e[1] < 0])

In [17]:
# Dijkstra testing
s = 0
t = 3
adj = {
    (0,1,1): 3,
    (0,2,2): 8,
    (1,0,3): 5,
    (1,2,4): 4,
    (1,3,5): 5,
    (2,1,6): 2,
    (2,3,7): 0,
    (3,1,8): 3
}
print(dijkstra(s,t,adj))

P=[(0, 1), (1, 2), (2, 3)]
({0: 0, 1: 3, 2: 7, 3: 7}, [(0, 1, 1), (1, 2, 4), (2, 3, 7)])
